# 전처리 과정

In [1]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')
PATH = "/content/drive/MyDrive/KUBIG"
train = pd.read_csv(PATH+"/train_project.csv")
test = pd.read_csv(PATH+"/test.csv")

Mounted at /content/drive


In [2]:
train = pd.read_csv(PATH+"/train_project.csv")


#완전히 중복되는 데이터 삭제
train=train[~train.drop(columns=['index']).duplicated(keep='first')]


# day_employed 양수값 0으로 변환
train['DAYS_EMPLOYED'].loc[train['DAYS_EMPLOYED']>0]=0

# occyp_type Unemployed 및 Laborers로 채우기
train['occyp_type'].loc[(train['DAYS_EMPLOYED']==0)&(train['occyp_type'].isna())]='Unemployed'
train['occyp_type']=train['occyp_type'].fillna('Laborers')
# train['occyp_type']=train['occyp_type'].fillna('Unknown')


# 의미없는 변수 제거 (index, FLAG_MOBIL)
train.drop(['index','FLAG_MOBIL'],axis=1,inplace=True)

# Label Encoding
#from sklearn.preprocessing import LabelEncoder
#classle = LabelEncoder()

#train['gender'] = classle.fit_transform(train['gender'].values)
#train['car'] = classle.fit_transform(train['car'].values)
#train['reality'] = classle.fit_transform(train['reality'].values)
#train['income_type'] = classle.fit_transform(train['income_type'].values)
#train['edu_type'] = classle.fit_transform(train['edu_type'].values)
#train['family_type'] = classle.fit_transform(train['family_type'].values)
#train['house_type'] = classle.fit_transform(train['house_type'].values)
#train['occyp_type'] = classle.fit_transform(train['occyp_type'].values)


# 자녀유무 변수
train['kids']=0
train['kids'] = np.where(train['child_num'] !=0, 1, 0)
train.drop(['child_num'],axis=1,inplace=True)


# income_total 로그 변환
# train['income_total']=np.log(train['income_total'])



# 카드 개수 변수
duplicate_rows = train[train.duplicated(subset=train.columns.difference(['begin_month', 'credit']), keep=False)]
duplicate_counts = duplicate_rows.groupby(duplicate_rows.columns.difference(['begin_month', 'credit']).tolist()).size().reset_index(name='count')
result = pd.merge(train, duplicate_counts, on=duplicate_rows.columns.difference(['begin_month', 'credit']).tolist(), how='left')
result['NumCards'] = result['count'].fillna(0).astype(int)
train = result
train.drop(['count'],axis=1,inplace=True)



# communication 변수 
train['communication'] = train['work_phone'] + train ['phone'] + train ['email']
train['communication'] = np.where(train['communication'] !=0, 1, 0)
train.drop(['work_phone', 'phone', 'email'], axis=1, inplace=True)



train = train [['gender', 'car', 'reality', 'income_total', 'income_type', 'edu_type',
       'family_type', 'house_type', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
       'communication', 'occyp_type', 'family_size',
       'begin_month','kids', 'NumCards', 'credit']]

# Pycaret 사용 (best5 ensemble)

In [ ]:
!pip install pycaret
!pip install catboost

## Setup

In [5]:
from pycaret.classification import *
from sklearn.metrics import log_loss

clf = setup(train, target = 'credit', train_size = 0.7, session_id=2023)

,Description,Value
0,session_id,2023
1,Target,credit
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(24823, 17)"
5,Missing Values,False
6,Numeric Features,6
7,Categorical Features,10
8,Ordinal Features,False
9,High Cardinality Features,False


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=[], target='credit',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strat...
                ('scaling', 'passthrough'), ('P_transform', 'passthrough'),
                ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
                ('cluste

In [6]:
#logloss 적용 및 predict_proba로 제출하기 위해 metric 추가
add_metric('logloss', 'LogLoss', log_loss, greater_is_better=False, target="pred_proba")

Name                                                           LogLoss
Display Name                                                   LogLoss
Score Function                   <function log_loss at 0x7fc924b85160>
Scorer               make_scorer(log_loss, greater_is_better=False,...
Target                                                      pred_proba
Args                                                                {}
Greater is Better                                                False
Multiclass                                                        True
Custom                                                            True
Name: logloss, dtype: object

## 모델별 수치

In [7]:
best5 = compare_models(fold = 5, sort = 'logloss', n_select = 5, exclude=['svm','ridge'])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.6937,0.6977,0.4320,0.6703,0.6230,0.2449,0.3140,0.7754,1.278
catboost,CatBoost Classifier,0.6925,0.6955,0.4418,0.6577,0.6303,0.2593,0.3129,0.7826,15.760
gbc,Gradient Boosting Classifier,0.6877,0.6629,0.4099,0.6636,0.6029,0.2072,0.2954,0.7977,10.182
lda,Linear Discriminant Analysis,0.6426,0.6149,0.3454,0.5320,0.5193,0.0359,0.0879,0.8633,0.304
nb,Naive Bayes,0.6424,0.6125,0.3418,0.5395,0.5127,0.0261,0.0812,0.8684,0.066
lr,Logistic Regression,0.6382,0.6040,0.3333,0.4073,0.4973,0.0000,0.0000,0.8708,2.736
dummy,Dummy Classifier,0.6382,0.5000,0.3333,0.4073,0.4973,0.0000,0.0000,0.8861,0.052
rf,Random Forest Classifier,0.6768,0.7159,0.5019,0.6485,0.6555,0.3175,0.3251,0.9576,3.758
ada,Ada Boost Classifier,0.6877,0.6402,0.4069,0.6649,0.6001,0.2019,0.2963,1.0786,0.764
et,Extra Trees Classifier,0.6497,0.6664,0.4798,0.6213,0.6296,0.2650,0.2707,2.4137,3.050


INFO:logs:create_model_container: 13
INFO:logs:master_model_container: 13
INFO:logs:display_container: 2
INFO:logs:[LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=2023, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0), <catboost.core.CatBoostClassifier object at 0x7fc902acce20>, GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_s

### Best 5 Ensemble (hyperparameter 튜닝 x)

In [8]:
# 튜닝하지 않은 경우
blended = blend_models(estimator_list = best5, fold = 5, optimize = 'logloss', method = 'soft')
pred_holdout = predict_model(blended)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
Fold,,,,,,,,
0,0.6896,0.6914,0.4112,0.7124,0.6040,0.2094,0.3028,0.7956
1,0.6955,0.6869,0.4166,0.7208,0.6110,0.2232,0.3246,0.7930
2,0.6901,0.6953,0.4118,0.6507,0.6057,0.2152,0.3023,0.7935
3,0.6932,0.6918,0.4186,0.6899,0.6119,0.2247,0.3139,0.7932
4,0.6901,0.6798,0.4084,0.6445,0.6026,0.2093,0.3043,0.8004
Mean,0.6917,0.6890,0.4133,0.6836,0.6070,0.2163,0.3096,0.7952
Std,0.0023,0.0053,0.0037,0.0312,0.0037,0.0066,0.0086,0.0028


INFO:logs:create_model_container: 14
INFO:logs:master_model_container: 14
INFO:logs:display_container: 3
INFO:logs:VotingClassifier(estimators=[('lightgbm',
                              LGBMClassifier(boosting_type='gbdt',
                                             class_weight=None,
                                             colsample_bytree=1.0,
                                             importance_type='split',
                                             learning_rate=0.1, max_depth=-1,
                                             min_child_samples=20,
                                             min_child_weight=0.001,
                                             min_split_gain=0.0,
                                             n_estimators=100, n_jobs=-1,
                                             num_leaves=31, objective=None,
                                             random_state=2023, reg_alpha=0.0,
                                             reg_lambda=0.0, silent

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,Voting Classifier,0.6932,0.6889,0.4169,0.716,0.6088,0.2237,0.3196,0.7957


### Best 5 Ensemble (hyperparameter 튜닝 o)

In [ ]:
tuned_best5 = [tune_model(i, optimize='logloss') for i in best5]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
Fold,,,,,,,,
0,0.6375,0.6077,0.3365,0.4856,0.5050,0.0122,0.0392,0.8736
1,0.6410,0.6119,0.3413,0.5150,0.5127,0.0265,0.0734,0.8690
2,0.6490,0.5841,0.3501,0.5887,0.5247,0.0483,0.1341,0.8773
3,0.6438,0.6260,0.3428,0.5527,0.5138,0.0294,0.0947,0.8614
4,0.6450,0.5947,0.3465,0.5543,0.5199,0.0386,0.1039,0.8737
5,0.6410,0.6331,0.3404,0.5410,0.5100,0.0204,0.0687,0.8657
6,0.6390,0.6455,0.3366,0.4941,0.5049,0.0123,0.0447,0.8597
7,0.6419,0.5908,0.3417,0.5455,0.5126,0.0238,0.0750,0.8756
8,0.6465,0.6089,0.3451,0.5829,0.5173,0.0352,0.1156,0.8612


INFO:logs:create_model_container: 31
INFO:logs:master_model_container: 31
INFO:logs:display_container: 10
INFO:logs:GaussianNB(priors=None, var_smoothing=1e-09)
INFO:logs:tune_model() succesfully completed......................................


In [ ]:
blend_tune = blend_models(estimator_list=tuned_best5, fold=10, optimize='logloss', method = 'soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
Fold,,,,,,,,
0,0.6916,0.7093,0.4234,0.6621,0.6159,0.2337,0.3062,0.7786
1,0.6922,0.7221,0.4215,0.6616,0.6146,0.2299,0.3095,0.7745
2,0.6864,0.7004,0.4127,0.6522,0.6052,0.2090,0.2909,0.7897
3,0.7071,0.7301,0.4426,0.6854,0.6365,0.2776,0.3550,0.7616
4,0.6939,0.6885,0.4284,0.6934,0.6199,0.2404,0.3137,0.7907
5,0.6974,0.7411,0.4286,0.6911,0.6217,0.2498,0.3243,0.7607
6,0.7006,0.7252,0.4362,0.6932,0.6286,0.2566,0.3358,0.7722
7,0.6978,0.6994,0.4322,0.6874,0.6246,0.2487,0.3261,0.7851
8,0.6897,0.6937,0.4182,0.6344,0.6121,0.2296,0.2994,0.7882


INFO:logs:create_model_container: 33
INFO:logs:master_model_container: 33
INFO:logs:display_container: 14
INFO:logs:VotingClassifier(estimators=[('lightgbm',
                              LGBMClassifier(bagging_fraction=0.7,
                                             bagging_freq=7,
                                             boosting_type='gbdt',
                                             class_weight=None,
                                             colsample_bytree=1.0,
                                             feature_fraction=0.5,
                                             importance_type='split',
                                             learning_rate=0.15, max_depth=-1,
                                             min_child_samples=100,
                                             min_child_weight=0.001,
                                             min_split_gain=0, n_estimators=90,
                                             n_jobs=-1, num_leaves=80,
            

In [ ]:
pred = predict_model(blend_tune)

INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=VotingClassifier(estimators=[('lightgbm',
                              LGBMClassifier(bagging_fraction=0.7,
                                             bagging_freq=7,
                                             boosting_type='gbdt',
                                             class_weight=None,
                                             colsample_bytree=1.0,
                                             feature_fraction=0.5,
                                             importance_type='split',
                                             learning_rate=0.15, max_depth=-1,
                                             min_child_samples=100,
                                             min_child_weight=0.001,
                                             min_split_gain=0, n_estimators=90,
                                             n_jobs=-1, num_leaves=80,
                                             objective=

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,Voting Classifier,0.6952,0.7108,0.4294,0.6805,0.62,0.2442,0.3227,0.7786


## 제출용

In [9]:
test = pd.read_csv(PATH+"/test.csv")


# day_employed 양수값 0으로 변환
test['DAYS_EMPLOYED'].loc[test['DAYS_EMPLOYED']>0]=0

# occyp_type Unemployed 및 Laborers로 채우기
test['occyp_type'].loc[(test['DAYS_EMPLOYED']==0)&(test['occyp_type'].isna())]='Unemployed'
test['occyp_type']=test['occyp_type'].fillna('Laborers')
# test['occyp_type']=test['occyp_type'].fillna('Unknown')


# 의미없는 변수 제거 (index, FLAG_MOBIL)
test.drop(['index','FLAG_MOBIL'],axis=1,inplace=True)

# Label Encoding
#from sklearn.preprocessing import LabelEncoder
#classle = LabelEncoder()

#test['gender'] = classle.fit_transform(test['gender'].values)
#test['car'] = classle.fit_transform(test['car'].values)
#test['reality'] = classle.fit_transform(test['reality'].values)
#test['income_type'] = classle.fit_transform(test['income_type'].values)
#test['edu_type'] = classle.fit_transform(test['edu_type'].values)
#test['family_type'] = classle.fit_transform(test['family_type'].values)
#test['house_type'] = classle.fit_transform(test['house_type'].values)
#test['occyp_type'] = classle.fit_transform(test['occyp_type'].values)


# 자녀유무 변수
test['kids']=0
test['kids'] = np.where(test['child_num'] !=0, 1, 0)
test.drop(['child_num'],axis=1,inplace=True)


# income_total 로그 변환
# test['income_total']=np.log(test['income_total'])



# 카드 개수 변수
duplicate_rows = test[test.duplicated(subset=test.columns.difference(['begin_month', 'credit']), keep=False)]
duplicate_counts = duplicate_rows.groupby(duplicate_rows.columns.difference(['begin_month', 'credit']).tolist()).size().reset_index(name='count')
result = pd.merge(test, duplicate_counts, on=duplicate_rows.columns.difference(['begin_month', 'credit']).tolist(), how='left')
result['NumCards'] = result['count'].fillna(0).astype(int)
test = result
test.drop(['count'],axis=1,inplace=True)



# communication 변수 
test['communication'] = test['work_phone'] + test ['phone'] + test ['email']
test['communication'] = np.where(test['communication'] !=0, 1, 0)
test.drop(['work_phone', 'phone', 'email'], axis=1, inplace=True)

test = test[['gender', 'car', 'reality', 'income_total', 'income_type', 'edu_type',
       'family_type', 'house_type', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
       'communication', 'occyp_type', 'family_size',
       'begin_month','kids', 'NumCards']]


In [10]:
final_model = finalize_model(blended)

INFO:logs:Initializing finalize_model()
INFO:logs:finalize_model(estimator=VotingClassifier(estimators=[('lightgbm',
                              LGBMClassifier(boosting_type='gbdt',
                                             class_weight=None,
                                             colsample_bytree=1.0,
                                             importance_type='split',
                                             learning_rate=0.1, max_depth=-1,
                                             min_child_samples=20,
                                             min_child_weight=0.001,
                                             min_split_gain=0.0,
                                             n_estimators=100, n_jobs=-1,
                                             num_leaves=31, objective=None,
                                             random_state=2023, reg_alpha=0.0,
                                             reg_lambda=0.0, silent='warn'...
                             

In [11]:
prep_pipe = get_config('prep_pipe')
prep_pipe.steps.append(['trained_model', final_model])
pred = prep_pipe.predict_proba(test)

INFO:logs:Initializing get_config()
INFO:logs:get_config(variable=prep_pipe)
INFO:logs:Global variable: prep_pipe returned as Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=[], target='credit',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strat...
                ('scaling', 'passthrough'), ('P_transform', 'passthrough'),
                ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
           

In [12]:
submit = pd.read_csv(PATH+'/sample_submission.csv')

i, j, k = [], [], []
for row in pred:
  i.append(row[0])
  j.append(row[1])
  k.append(row[2])

len(i), len(j), len(k)

(10000, 10000, 10000)

In [13]:
submit['0'] = i
submit['1'] = j
submit['2'] = k
submit.head(10)

submit.to_csv(PATH+"/submisison.csv",index=False)

,index,0,1,2
0,26457,0.096952,0.169502,0.733546
1,26458,0.139588,0.135346,0.725066
2,26459,0.099928,0.138228,0.761844
3,26460,0.129148,0.129064,0.741788
4,26461,0.116557,0.203561,0.679881
5,26462,0.127513,0.269195,0.603291
6,26463,0.202493,0.515008,0.282499
7,26464,0.142215,0.254527,0.603258
8,26465,0.091331,0.141420,0.767249
9,26466,0.087480,0.203266,0.709254
